In [2]:
import pandas as pd
from google.colab import drive
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

# Mount Google Drive
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Stroke_data.csv"

# Load dữ liệu
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.drop(columns=['id'], inplace=True)
    return df

# Tính BMI từ chiều cao và cân nặng
def calculate_bmi(weight, height):
    height_m = height / 100  # Chuyển đổi cm sang mét
    return weight / (height_m ** 2)

# Tiền xử lý dữ liệu
def preprocess_data(df):
    imputer = SimpleImputer(strategy='mean')
    df['bmi'] = imputer.fit_transform(df[['bmi']])

    label_encoders = {}
    categorical_columns = ['gender', 'ever_married', 'Residence_type']
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    df = pd.get_dummies(df, columns=['work_type', 'smoking_status'], drop_first=True)
    return df, label_encoders

# Cân bằng dữ liệu
def balance_data(df):
    df_majority = df[df.stroke == 0]
    df_minority = df[df.stroke == 1]
    df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
    df_balanced = pd.concat([df_majority_downsampled, df_minority])
    return df_balanced

# Tối ưu Hyperparameters và huấn luyện mô hình
def train_best_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [5, 10, 15],
        'min_samples_split': [2, 5, 10]
    }
    rf = RandomForestClassifier()
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    print(f"Best Random Forest Model - Accuracy: {acc:.4f}, AUC-ROC: {auc:.4f}")

    return best_model

# Dự đoán tỷ lệ đột quỵ
def predict_stroke(model, input_data, label_encoders, feature_columns):
    input_df = pd.DataFrame([input_data])
    for col in ['gender', 'ever_married', 'Residence_type']:
        input_df[col] = label_encoders[col].transform(input_df[col])
    input_df = pd.get_dummies(input_df, columns=['work_type', 'smoking_status'], drop_first=True)
    input_df = input_df.reindex(columns=feature_columns, fill_value=0)
    return model.predict_proba(input_df)[:, 1][0] * 100

# Chạy chương trình
if __name__ == "__main__":
    df = load_data(file_path)
    df, label_encoders = preprocess_data(df)
    df_balanced = balance_data(df)

    X = df_balanced.drop(columns=['stroke'])
    y = df_balanced['stroke']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    best_model = train_best_model(X_train, y_train, X_test, y_test)

    while True:
        print("\nMenu:")
        print("1. Bắt đầu nhập dữ liệu người dùng")
        print("2. Nhập dữ liệu người dùng tiếp theo")
        print("3. Dừng chương trình")
        choice = input("Chọn một tùy chọn: ")

        if choice == "1" or choice == "2":
            gender = input("Nhập giới tính (Male/Female): ")
            age = int(input("Nhập tuổi: "))
            hypertension = int(input("Có bị cao huyết áp không? (0: Không, 1: Có): "))
            heart_disease = int(input("Có bệnh tim không? (0: Không, 1: Có): "))
            ever_married = input("Đã kết hôn chưa? (Yes/No): ")
            work_type = input("Loại công việc (Private/Self-employed/Govt_job/Children/Never_worked): ")
            residence_type = input("Loại nơi ở (Urban/Rural): ")
            avg_glucose_level = float(input("Nhập mức glucose trung bình: "))
            weight = float(input("Nhập cân nặng (kg): "))
            height = float(input("Nhập chiều cao (cm): "))
            bmi = calculate_bmi(weight, height)
            smoking_status = input("Tình trạng hút thuốc (formerly smoked/never smoked/smokes): ")

            sample_input = {
                "gender": gender,
                "age": age,
                "hypertension": hypertension,
                "heart_disease": heart_disease,
                "ever_married": ever_married,
                "work_type": work_type,
                "Residence_type": residence_type,
                "avg_glucose_level": avg_glucose_level,
                "bmi": bmi,
                "smoking_status": smoking_status
            }

            stroke_probability = predict_stroke(best_model, sample_input, label_encoders, X.columns)
            print(f"Tỷ lệ mắc đột quỵ: {stroke_probability:.2f}%")
        elif choice == "3":
            print("Chương trình kết thúc.")
            break
        else:
            print("Lựa chọn không hợp lệ, vui lòng chọn lại.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Best Random Forest Model - Accuracy: 0.7500, AUC-ROC: 0.7521

Menu:
1. Bắt đầu nhập dữ liệu người dùng
2. Nhập dữ liệu người dùng tiếp theo
3. Dừng chương trình
Chọn một tùy chọn: 1
Nhập giới tính (Male/Female): Male
Nhập tuổi: 20
Có bị cao huyết áp không? (0: Không, 1: Có): 0
Có bệnh tim không? (0: Không, 1: Có): 0
Đã kết hôn chưa? (Yes/No): No
Loại công việc (Private/Self-employed/Govt_job/Children/Never_worked): Never_worked
Loại nơi ở (Urban/Rural): Urban
Nhập mức glucose trung bình: 120
Nhập cân nặng (kg): 85
Nhập chiều cao (cm): 170
Tình trạng hút thuốc (formerly smoked/never smoked/smokes): never smoked
Tỷ lệ mắc đột quỵ: 18.83%

Menu:
1. Bắt đầu nhập dữ liệu người dùng
2. Nhập dữ liệu người dùng tiếp theo
3. Dừng chương trình
Chọn một tùy chọn: 3
Chương trình kết thúc.
